# Assignment 1: Decision Trees (10 marks)

Student Name: `Arya Araban`

Student ID: `1439683`

## General info

<b>Due date</b>: 

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -10% per day up to 5 days (both weekdays and weekends count)
<ul>
    <li>one day late, -1.0 ;</li>
    <li>two days late, -2.0;</li>
    <li>three days late, -3.0;</li>
    <li>four days late, -4.0;</li>
    <li>five days late, -5.0;</li>
</ul>

<b>Marks</b>: This assignment will be marked out of 10, and make up 10% of your overall mark for this subject.

<b>Materials</b>: See <a href="https://canvas.lms.unimelb.edu.au/courses/151131/pages/python-and-jupyter-notebooks?module_item_id=4532241">[Using Jupyter Notebook and Python page]</a> on Canvas (under Modules> Coding Resources) for information on the basic setup required for this class, including an iPython notebook viewer and the python packages `numpy` and `pprint`. You can use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>.  


<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods and answers.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on Canvas> Assignments>A ssignment1. We recommend you check it regularly.

<b>Academic misconduct</b>: This assignment is an individual task, and so reuse of code or other instances of clear influence will be considered cheating. Please check the <a href="https://canvas.lms.unimelb.edu.au/courses/124196/modules#module_662096">CIS Academic Honesty training</a> for more information. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where collusion or plagiarism are deemed to have taken place. Content produced by an AI (including, but not limited to ChatGPT) is not your own work, and submitting such content will be treated as a case of academic misconduct, in line with the <a href="https://academicintegrity.unimelb.edu.au/plagiarism-and-collusion/artificial-intelligence-tools-and-technologies"> University's policy</a>.

**IMPORTANT**

Please carefully read and fill out the <b>Authorship Declaration</b> form at the bottom of the page. Failure to fill out this form results in the following deductions: 
<UL TYPE=”square”>
<LI>Missing Authorship Declaration at the bottom of the page, -1.0
<LI>Incomplete or unsigned Authorship Declaration at the bottom of the page, -0.5
</UL>

## Overview

In this assignment, you will apply the Decision Tree (DT) classification algorithm to a real-world machine learning dataset. Specifically, you will predict the type of a given bridge using a diverse set of features, including its material, length, number of lanes, and other structural properties. We use a modified version of the publicly available <a href="https://archive.ics.uci.edu/ml/datasets/Pittsburgh+Bridges"> Pittsburgh Bridges Data Set</a>. (click the link for more information on all features and values).

In the **Data Preparation** section, you will read the dataset into a data frame and perform data preprocessing (Q1.1). You will also need to develop code to convert the features to numeric values (Q1.2). Then, the data will be divided into two parts: Train and Test.

In the **Model Training** section, we provide you with a number of helper functions that will be incorporated into the main Decision Tree function (Q2-8). Your task is to read and use these functions and explain the exact role of each helper function in developing a Decision Tree. You must provide a meaningful and descriptive name for each function that matches its specific role in training a Decision Tree model (e.g., count_labels, calculate_entropy, etc.).

You can observe the use of each of these functions in the Main Decision Tree Algorithm, which you can use to print your trained tree. We have also provided the code for navigating a trained tree and using it to predict the labels for the test dataset.

In the **Classification and Evaluation** section, you will test and evaluate your model. You will implement functions to calculate the accuracy of the model predictions (Q9). Finally, you will train a tree and use it to predict the labels for the test set and see the accuracy of your model (Q10).

Q11 concludes with analytical questions in which you will critically analyze the behavior of your decision tree in different situations.


In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import numpy as np
import pandas as pd

from collections import Counter
from pprint import pprint

## Data Preparation [1 mark]

The Bridges dataset consists of 108 bridges. Each bridge is defined by 11 features and 1 label. These features are `Index`, `River`, `Erected`, `Purpose`, `Length`, `Lanes`, `Clear`, `T-OR-D`, `Material`, `Span` and `Rel-L`. Note that the first feature is only an index and not useful as a feature for classification. 


#### Q1.1 Reading the input file and data preprocessing (0.25 marks)

This code should read the input file into a `pandas` dataframe. You should remove the first feature because it is only an id and perform some preprocessing. In the preprocessing phase you are going to replace all missing values (`?`) with the most frequent (most common) value of that feature. 

In [22]:
# read the input file
df = pd.read_csv("drive/MyDrive/datasets/bridges.data.csv")


############ YOUR CODE HERE ##############

# First, drop the index column

df.drop('Index', axis=1, inplace = True)

# Second, replace all missing values with the most frequet value of the features

for col_name in df.columns:
    mode = df[col_name].mode()[0]  # find the most frequent value 
    df[col_name].replace('?',mode, inplace=True)  # replace '?' values with frequent value
    
############## TEST IT YOURSELF ###############
assert df['Length'][0] == 'MEDIUM'
assert df['Lanes'][24] == '2'

print(df.head())

  River Erected   Purpose  Length Lanes Clear   T-OR-D Material    Span Rel-L  \
0     M  CRAFTS   HIGHWAY  MEDIUM     2     N  THROUGH     WOOD   SHORT     S   
1     A  CRAFTS  AQUEDUCT  MEDIUM     1     N     DECK     WOOD  MEDIUM     S   
2     O  MODERN   HIGHWAY  MEDIUM     2     G  THROUGH    STEEL  MEDIUM     F   
3     O  MATURE   HIGHWAY  MEDIUM     2     G  THROUGH    STEEL    LONG   S-F   
4     O  MODERN   HIGHWAY  MEDIUM     2     G  THROUGH    STEEL    LONG     F   

      Label  
0      WOOD  
1      WOOD  
2      ARCH  
3  CANTILEV  
4    CONT-T  


#### Q1.2 Convert Categorical features to Numeric (0.5 marks)
In order to simplify our calculations we are going to transform the features (columns) with categorical values to numbers. For example, for the feature `River` there are 3 possible values: 'A', 'M' and '0'. 

**NOTE:** Sort them alphabatically and assign sequencial integer values starting from 0. For `River` we will have: 'A' = 0, 'M' = 1, 'O' = 2.


In [23]:
from pandas.core.dtypes.cast import dict_compat
############ YOUR CODE HERE ##############

replace_dict = {}
for col_name in df.columns:
    categories = sorted(df[col_name].unique())  
    replace_dict[col_name] = {cat: i for i, cat in enumerate(categories)}  

replace_dict.pop('Label') # because we don't want the Label to be Numeric.

df = df.replace(replace_dict) # replace everything in the dataframe with the obtained dictionary. 
##########################################



In [24]:
df

,River,Erected,Purpose,Length,Lanes,Clear,T-OR-D,Material,Span,Rel-L,Label
0,1,0,1,1,1,1,1,2,2,1,WOOD
1,0,0,0,1,0,1,0,2,1,1,WOOD
2,2,3,1,1,1,0,1,1,1,0,ARCH
3,2,2,1,1,1,0,1,1,0,2,CANTILEV
4,2,3,1,1,1,0,1,1,0,0,CONT-T
...,...,...,...,...,...,...,...,...,...,...,...
103,1,2,2,1,1,0,1,1,1,1,SIMPLE-T
104,1,3,2,1,1,0,1,1,1,0,SIMPLE-T
105,1,3,1,1,1,0,1,1,1,1,ARCH
106,1,3,1,2,2,0,1,1,1,0,CONT-T


#### Q1.3 Split the data into a Train and Test Set (0.25 marks)

The first 85 instances should be used for training and the last 23 instances for testing. NO SHUFFLING IS ALLOWED!

In [25]:
# the first 85 instances should be used for training and the last 23 instances for testing
# Don't shuffle the data!
split_on = 85

train_df = df.iloc[:split_on,:]
test_df = df.iloc[split_on:,:]

assert(len(train_df)==85)
assert(len(test_df)==23)

In [26]:
train_df

,River,Erected,Purpose,Length,Lanes,Clear,T-OR-D,Material,Span,Rel-L,Label
0,1,0,1,1,1,1,1,2,2,1,WOOD
1,0,0,0,1,0,1,0,2,1,1,WOOD
2,2,3,1,1,1,0,1,1,1,0,ARCH
3,2,2,1,1,1,0,1,1,0,2,CANTILEV
4,2,3,1,1,1,0,1,1,0,0,CONT-T
...,...,...,...,...,...,...,...,...,...,...,...
80,1,2,1,0,1,0,0,1,1,2,CANTILEV
81,0,2,1,0,2,0,0,1,1,0,ARCH
82,1,2,1,1,2,0,1,1,0,0,SUSPEN
83,2,2,1,1,2,1,1,1,0,0,ARCH


## Training a Decision Tree [5 marks]

### Helper Functions

For answers to the questions Q2--Q7 we expect about 3-5 sentences, and for Q8 about 6-10 sentences.


**Q2. Provide an descriptive name for `func1` and explain the role of this function in training/building a Decision Tree. (0.5 marks)**

name: over_one_unique
</br>
</br>

This function checks to see whether or not the last column of a 2d array has over one unique value. </br>
If it does, then 'True' is returned, otherwise 'False' is returned.

Given the current data we have at hand, the assessed column will be contain the classes assigned to each row.

This can be a crucial function for a Decision Tree, since in Decision Trees, we split the data based on the values of its features, and at each node of the tree it has to be checked if the output class is the same for all the remaining instances. If so, there's no need to expand that specific node any further and it will become a leaf node which has the value of the only remaining class. Otherwise, it needs to be expanded.
</br>
</br>





In [27]:
#This function receives a 2D array of values the passed instances should have a format such as:
#    [[1 0 1 1 1 1 1 2 2 1 'WOOD'] 
#     [0 0 0 1 0 1 0 2 1 1 'WOOD']
#     [2 3 1 1 1 0 1 1 1 0 'ARCH']
#     [2 2 1 1 1 0 1 1 0 2 'CANTILEV'] 
#     [2 3 1 1 1 0 1 1 0 0 'CONT-T']
#      ...] -- is a numpy array

def func1(data): 
    # the input `data` is a 2D array as illustrated above
    # the output `answer` is a BOOLEAN (True or False)
    
    column = data[:, -1]
    values = np.unique(column)

    if len(values) == 1:
        return True
    else:
        return False
   

**Q3. Provide a descriptive name for `func2` and explain the role of this function in training/building a Decision Tree. (0.5 marks)**


name: get_majority_class
</br>
</br>

This function returns the name of the most frequent class in a given data subset. It can be useful for assigning a class to a node in a decision tree, causing it become a leaf node.
</br>
The reason we need this in a Decision Tree is that if we only consider nodes with 100% purity (or 0 entropy) as a leaf node, our Decision Tree might become too big and cause overfitting. Therefore, We might create some limits on whether a node can expand or not (For example, we might have a set maximum depth, or in the case of the ID3 algorithm we can make a slight modification to only expand nodes which have the highest information gain being above a specified threshold), and if we can't expand the node anymore, we can use this function to turn it into a leaf node based by performing a class majority voting among the instances currently contained in the node. 
</br>
</br>
</br>
</br>





In [28]:
def func2(data):
    # the input `data` is a 2D array as illustrated above
    # the output is a STRING 
    
    column = data[:, -1] 
    
    values, counts = np.unique(column, return_counts=True)
    index = counts.argmax()
    name = values[index]
    
    return name

**Q4. (1) Provide a descriptive name for `func3` and two parameters `name` and `value`. You need to explain what is the role of this function (and these two parametrs) in training/building a Decision Tree.  
(2) What are `data1` and `data2` in the context of a Decision Tree? (1 mark)**  

(1) name: split_by_value_of_feature
</br>
</br>
This function splits the current data of a node based on the value of a specified feature into two halves.
</br>
When creating a Decision Tree, we usually perform binary splitting on each node as opposed to non-binary splitting. Therfore, since we create decision trees in a greedy top-to-bottom manner, this function can help in splitting our data into two halves at each node.
</br>
</br>
(2) Data1 and Data2 are the two subsets of data which are obtained after the splitting is done. Data1 contains all the instances where the feature value is less than the specified value, and data2 contains all the instances where the feature value is greater than or equal to the specified value.
</br>
</br>
</br>





In [29]:
def func3(data, array, feature_name, feature_value):
    # the input `data` is a 2D array 
    # the input `array` is the list of names
    # the input `name` specifies a member in the array
    # the input `value` specifies a value for that array member
    # the outputs `data1` and `data2` are 2D arrays 

    index = array.get_loc(feature_name) # array is the list of all unique feature names in order, here the index of a specific feature is taken
    data1 = data[data[:,index] < feature_value] # use that feature in order to perform splitting based on an inputted splitting value
    data2 = data[data[:,index] >= feature_value]
    
    return data1, data2

**Q5. Provide a descriptive name for `func4` and explain what is the output of this function? (0.5 marks)**


name: entropy_calculation
</br>
</br>

This function calculates entropy. The lower the entropy, the more pure the node will be. 

In ID3 we measure how much entropy is reduced after splitting on a specified feature and feature value in order to calculate the information gain for that specific feature and splitting-value. After trying this for different possibilites, the feature and splitting-value which contains the highest information gain is selected to perform splitting on.
</br>
</br>
</br>



In [30]:
def func4(data):
    # the input `data` is a 2D array 
    # the output is a single real-valued number
    
    column = data[:, -1]
    _, counts = np.unique(column, return_counts=True)

    x = counts / counts.sum()  
    y = sum(x * -np.log2(x))
    
    return y

**Q6. Provide a descriptive name for `func5` and explain what is the output of this function? (0.5 marks)**

name: weighted_entropy_calculation
</br>
</br>
This function calculates the average of the weighted entropy of the two resulting nodes obtained after performing splitting. 

The entropy of each of the two nodes is calculated seperately. The obtained enropies are then multiplied by their corresponding weights (which is based on the number of instances in each node) and finally the two resulting numbers are summed together and divided by the total number of instances. 

This output can then be compared to the entropy of the parent node in order to calculate the information gain.
</br>
</br>
</br>



In [62]:
def func5(data1, data2):
    # the inputs `data1` and `data2` are both 2D arrays
    # the output is a single real-valued number

    n = len(data1) + len(data2)
    x1 = len(data1) / n
    x2 = len(data2) / n

    y =  (x1 * func4(data1) + x2 * func4(data2))
    
    return y

**Q7. Provide a descriptive name for `func6` and explain what is the output of this function? (0.5 marks)**

name: information_gain_calculation
</br>
</br>
This function outputs the information gain obtained by splitting the input (data0) into two subsets (data1 and data2).

Information gain is calculated by subtracting the average of the weighted entropy of these subsets (obtained with "func5") from the original entropy of the parent node (obtained with the "entropy_calculation" function). Note that I've explained how information gain is used in my answer to Q5. 
</br>
</br>
</br>


In [32]:
def func6(data0, data1, data2):
    # the inputs `data0`, `data1` and `data2` are all 2D arrays
    # the output is a single real-valued number
    
    x0 = func4(data0)
    y = x0 - func5(data1, data2)
    
    return y

**Q8. Explain the following variables in `func7` and provide a descriptive name for them. (2 marks)</br>**
**1. x </br> 2. Y </br> 3. y1 </br> 4. y2**



1-
x ~~ name: information_gain

this variable is the information gain obtained by splitting the data based on a given feature name and feature value.
</br>

2- Y ~~ name: highest_information_gain

This variable holds the highest information gain among all the splits for the features / splitting-values we have. We note that the information gain is obtained iteravely and this variable is only updated if the current gain has higher value than the previous highest gain. This results in the variable holding the highest possible gain after the inner loop terminates.
</br>

3- y1 ~~ name: best_feature_name

This variable is the name of the feature which performing splitting on gives the highest observed information gain.

</br>
4- y2 ~~ name: best_feature_value

This variable is the best value to split on resulting in the highest observed information gain.

</br>
function name: pick_best_split
</br>
This function outputs the best seen feature name and splitting-value which is used in order to split the data into two subsets that maximize the information gain. 

I've added comments in order to further explain what happens in each step of the code.
</br>
</br>
</br>



In [33]:
def func7(data,array):
    # note array in form of ['feat1', 'feat2', 'feat3' etc..]
    first_iteration = True
    
    for a in array[:-1]: # only get the features and not the label 

        values = np.unique(data[:,array.get_loc(a)]) # get all the possible values for the specific feature 'a'
        
        for v in values: # perform splitting on those values!
            
            data1, data2 =  func3(data, array, a, v)
            
            x = func6(data, data1, data2)  
            
            if first_iteration or x >= Y: # compare with the previous best split, regardless of it being from the same feature or not
                first_iteration = False  # If the info gain is better than the current best seen, then update!
                Y = x
                y1 = a
                y2 = v
    
    return y1, y2
            

### Main Decision Tree Algorithm
This is the recursive part of developing the Decision Tree. If you have developed all the previous parts correctly this function will make a Decision Tree for the passed dataset. We can set the maximum depth of the tree (`max_depth`) and number of samples that we would stop spliting a node (`min_samples`). <br>


In [42]:
def decision_tree_algorithm(data, array, max_depth, counter=0, min_samples=2):
    
    # We first write the case where the recursive algorithm reach to an stop.
    # The algorithm stops if func1 returns `true` 
    #    or instances left in the node is less than define minimum samples (e.g., 2) 
    #    or we have reached the maximum length of the tree
    
#     print(counter, max_depth, func1(data), len(data) < min_samples, counter == max_depth)
       
    if (func1(data)) or (len(data) < min_samples) or (counter == max_depth): 
#         print('break')
        y = func2(data)
        return y
    
    # recursive part
    else:
#         print('recursive')
        
        counter += 1
        
        name, value = func7(data,array)
        data1, data2 = func3(data, array, name, value)
        
        if (len(data1) == 0) or (len(data2) == 0):
            y = func2(data)
            return y
    
        #instanciate the tree
        node = "{} < {}".format(name, value)
        sub_tree = {node: []}
              
        #develop the sub-trees (recursion)
        left_child = decision_tree_algorithm(data=data1, array=array, max_depth=max_depth, counter=counter)
        right_child = decision_tree_algorithm(data=data2, array=array, max_depth=max_depth, counter=counter)
        
        
        if left_child == right_child:
            sub_tree = right_child
        else:
            sub_tree[node].append(left_child)
            sub_tree[node].append(right_child)
        
        return sub_tree   
         

In [43]:
############## TEST IT YOURSELF ###############
data = train_df
tree = decision_tree_algorithm(data=data.values, array=data.columns, max_depth=3, counter=0)

pprint(tree)

{'Material < 2': [{'Purpose < 2': [{'Rel-L < 2': ['SIMPLE-T', 'SUSPEN']},
                                   'SIMPLE-T']},
                  'WOOD']}


With the max_depth of 3, your tree would be:

`Material < 2`   ------- `Purpose < 2`  ------ `Rel-L < 2`------- **SIMPLE-T** <br>
$\;\;\;\;|\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;|\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;|$------  **SUSPEN**<br>
$\;\;\;\;$|----- **WOOD** $\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$|------  **SIMPLE-T** 


# Classification and Evaluation [1 mark]

Now that we have built our decision tree, we are ready to use it to predict a label for unseen instances. We provided the code to do this, below.

In [44]:
def predict_one_instance(example, tree):
    
    # tree is just a root node
    if not isinstance(tree, dict):
        return tree
    
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")

    # ask question
    if (example[feature_name] < float(value)):
        answer = tree[question][0]
    else:
        answer = tree[question][1]

    # base case (the answer in not a dictionary)
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return predict_one_instance(example, residual_tree)


In [37]:
def make_predictions(df, tree):
    
    if len(df) != 0:
        predictions = df.apply(predict_one_instance, args=(tree,), axis=1)
    else:
        # "df.apply()"" with empty dataframe returns an empty dataframe,
        # but "predictions" should be a series instead
        predictions = pd.Series()
        
    return predictions

**Q9. Write a function to calculate accuracy (0.5 marks)**

This function receives a dataframe and predictions of a trained tree (on that dataset) and should calculate the accuracy of the prdictions for the passed dataframe.

**NOTE:** You are NOT allowed to use Python accuracy methods here. 

In [58]:
def calculate_accuracy(df, predictions):
    
    ########### YOUR CODE HERE ##############
    # we know that accuracy is the number of correct predictions made divided by the total number of predictions made.
    actual = np.array(df[df.columns[-1]])

    correct = (actual == predictions) 
    accuracy = correct.sum() / len(actual) # or len(predics), doesn't matter.
    #########################################
    
    return accuracy

**Q10. Use the Tree (0.5 marks)**

Using all the developed functions above build a tree using the **Training set** (train_df) (max_depth = 2) and then use it to predict labels for the **Test set** (test_df). Then print the accuracy of your tree for the test set.

In [59]:
#############  YOUR CODE HERE ##############
#vals = np.unique(np.array(df[df.columns[-1]]))

tree = decision_tree_algorithm(data=train_df.values, array=train_df.columns, max_depth=2, counter=0)

predics = make_predictions(test_df,tree)

accuracy = calculate_accuracy(test_df, predics)

###########################################

print(accuracy) 

0.34782608695652173


## Q11. Analytical questions [2 marks]

Answer each of the following subquestions with a text answer of 3-5 sentences, using the results you obtained in the previous sections. You don't need to develop new functions. You might need to re-run and reuse some of the functions you already implemented in the ### CODE ### cells above.

### Q11.1 Manipulating the features (1.5 marks)

Let's see if changes in the number of features changes the tree results.

**(A)** Train a tree (T1) with max_depth = 2, using the train_df. Use T1 to predict the labels for both train_df and test_df and print the accuracy for both datasets.

**(B)** For both train_df and test_df remove (drop) the `Material` feature. Then train another tree (T2) with max_depth = 2 using the new train set (no_mat_train_df). Use T2 to predict labels for new test and train set and check the accuracy of the predictions. Are the results different from what they were in part A? If yes, explain why. 

In [60]:
T1 = decision_tree_algorithm(data=train_df.values, array=train_df.columns, max_depth=2, counter=0)

train_predics = make_predictions(train_df,T1)
test_predics = make_predictions(test_df,T1)


train_accuracy = calculate_accuracy(train_df, train_predics)

test_accuracy = calculate_accuracy(test_df, test_predics)

###########################################

print(f"accuracy of decision tree on training set: {train_accuracy}") 
print(f"accuracy of decision tree on test set: {test_accuracy}") 

accuracy of decision tree on training set: 0.611764705882353
accuracy of decision tree on test set: 0.34782608695652173


In [61]:
new_train_df = train_df.drop('Material', axis=1)
new_test_df = test_df.drop('Material', axis=1)

T1 = decision_tree_algorithm(data=new_train_df.values, array=new_train_df.columns, max_depth=2, counter=0)


train_predics = make_predictions(new_train_df,T1)
test_predics = make_predictions(new_test_df,T1)


train_accuracy = calculate_accuracy(new_train_df, train_predics)

test_accuracy = calculate_accuracy(new_test_df, test_predics)

###########################################

print(f"accuracy of decision tree on training set: {train_accuracy}") 
print(f"accuracy of decision tree on test set: {test_accuracy}") 

accuracy of decision tree on training set: 0.5882352941176471
accuracy of decision tree on test set: 0.43478260869565216


It is apparent that by dropping the 'Material' feature column, we are getting better results predicting the test data as opposed to not dropping the column (around a 26% increase at better predicting labels). 

The main reason for this can be that by removing this feature, we have successfuly reduced the complexity of the decision tree. This can cause the tree to "overfit" less. Overfitting can happen when a decision tree is complex and fits the training data a bit too closely, thus making it perform worse on non-seen data. 

It is due to this exact reason that we see slighty worse prediction accuracy on the training set, since the decision tree now has less information to use for data causing it to be less specialized towards the training data.
</br>
</br>
</br>



### Q11.2 Decision tree complexity (1 mark)

**(A)** Using the tree you generated in Q10 (name it `little_tree`), find the accuracy of the tree for predicting the labels for the **training set**. Do you notice a difference between train and test accuracy? If so, discuss possible reasons. 

**(B)** Now change the max_depth of the decision tree to 10 and train another tree (name it `big_tree`). Now use this new tree to predict the labels for test and train sets. Describe and explain any change in the results you notice compared to your tree of depth 2.</br>


In [64]:
little_tree = decision_tree_algorithm(data=train_df.values, array=train_df.columns, max_depth=2, counter=0) # exactly the same as the tree in Q10

train_predics = make_predictions(train_df, little_tree)
test_predics = make_predictions(test_df, little_tree)

train_accuracy = calculate_accuracy(train_df, train_predics)

test_accuracy = calculate_accuracy(test_df, test_predics)

print(f"accuracy of decision tree on training set: {train_accuracy}") 
print(f"accuracy of decision tree on test set: {test_accuracy}") 


accuracy of decision tree on training set: 0.611764705882353
accuracy of decision tree on test set: 0.34782608695652173


In [75]:
big_tree = decision_tree_algorithm(data=train_df.values, array=train_df.columns, max_depth=10, counter=0) 

train_predics = make_predictions(train_df, big_tree)
test_predics = make_predictions(test_df, big_tree)

train_accuracy = calculate_accuracy(train_df, train_predics)

test_accuracy = calculate_accuracy(test_df, test_predics)

print(f"accuracy of decision tree on training set: {train_accuracy}") 
print(f"accuracy of decision tree on test set: {test_accuracy}") 

accuracy of decision tree on training set: 0.9411764705882353
accuracy of decision tree on test set: 0.34782608695652173


*(A)*
It is apparent that the accuracy obtained by predicting the training data is higher than predicting the test data. The reason for this is that when we create a decision tree, the tree is specifically designed to fit the training data. After which we evaluate how well this tree performs on seeing unseen data
(which we know as the test data). When designing a decision tree and its rules (such as max_depth, number of features, etc..) we have to be careful to choose the parameters in such a way that we prevent overfitting as best as we can.
<br/>
<br/>
<br/>

*(B)* When we increase the max depth of a decision tree, we are basically allowing the tree to become more complex by learning more of the relationships between the features in the training data. This can cause the tree to give us much better accuracy for predicting labels of the training data, which is exactly what we see in this example. It is noteworthy that the more we increase the depth of a decision tree the more likely we are to create a model which overfits the training data and performs worse on unseen data. In this example we are getting the exact same accuracy results when predicting the test data on the two different tree lengths. The reason for this can be that when training a decision tree with a maximum depth of 2 we are underfitting the data, but with a maximum depth of 10, we are overfitting the data, thus causing the same weak result. So as an example, if we select the maximum depth of the tree to be 4,  the accuracy of predicting the test data will be 0.47 which is much better than our previous result.

# Authorship Declaration:

   (1) I certify that the program contained in this submission is completely
   my own individual work, except where explicitly noted by comments that
   provide details otherwise.  I understand that work that has been developed
   by another student, or by me in collaboration with other students,
   or by non-students as a result of request, solicitation, or payment,
   may not be submitted for assessment in this subject. The same holds for AI models including, but not limited to, ChatGPT. I understand that
   submitting for assessment work developed by or in collaboration with
   other students or non-students constitutes Academic Misconduct, and
   may be penalized by mark deductions, or by other penalties determined
   via the University of Melbourne Academic Honesty Policy, as described
   at https://academicintegrity.unimelb.edu.au.

   (2) I also certify that I have not provided a copy of this work in either
   softcopy or hardcopy or any other form to any other student, and nor will
   I do so until after the marks are released. I understand that providing
   my work to other students, regardless of my intention or any undertakings
   made to me by that other student, is also Academic Misconduct.

   (3) I further understand that providing a copy of the assignment
   specification to any form of code authoring or assignment tutoring
   service, or drawing the attention of others to such services and code
   that may have been made available via such a service, may be regarded
   as Student General Misconduct (interfering with the teaching activities
   of the University and/or inciting others to commit Academic Misconduct).
   I understand that an allegation of Student General Misconduct may arise
   regardless of whether or not I personally make use of such solutions
   or sought benefit from such actions.

   <b>Signed by</b>: Arya Araban - 1439683
   
   <b>Dated</b>: [Enter the date that you "signed" the declaration]